#Exerici 1: Creació d'individus
##Definició del genotip i fenotip
    Genotip -> Matriu de 16 files (quarks) i 28 columnes (orbital)
    Fenotip -> Comportament resultant per tant en aquest cas cal tenir en compte: 
                - Que cap orbital superi el màxim d'influències permeses.
                - Que cap quark influeixi més de dos orbitals consecutius.

#Estructura de l'ADN
La matriu és binària per representar la influència o no dels quarks sobre els orbitals.
D'aquesta manera les operaciones genètiques posteriors són senzilles i es pot avaluar ràpidament si el resultat obtingut és correcte.

#Principi de variació
La inicialització és aleatòria però controlada per tal de garantir que es compleixin les normes de no influenciar més de dos orbitals consecutius i de no sobrecarregar els orbitals.
Mitjançant mutacions amb probabilitat es canvien els 0 a 1 i els 1 a 0 comprovant que no es trenquin les restriccions. Per tant, mantenint una diversitat genètica en la població inicial i permeten variació a altres configuracions per a les generacions futures evitant l'estancament en un o una sèrie de patrons. 

In [2]:
import numpy as np
import pygame
import random
def create_adn():

    initial = np.zeros((16,28), dtype=int)
    orbitals_count = np.zeros(28, dtype=int)
    max_influence = [8,8,8,8,10,6,4] * 4

    for quark in range(16):
        for orbital in range(28):
            influence = np.random.randint(0, 2)

            influence_available = True

            if orbital >= 2 and initial[quark, orbital-1] == 1 and initial[quark, orbital-2] == 1:
                influence_available = False
            if influence == 1 and orbitals_count[orbital] >= max_influence[orbital]:
                influence_available = False
            if influence == 1 and influence_available:
                orbitals_count[orbital] += 1
                initial[quark, orbital] = 1
    
    return initial

initial = create_adn()
print(initial)

pygame 2.6.1 (SDL 2.28.4, Python 3.13.3)
Hello from the pygame community. https://www.pygame.org/contribute.html
[[1 1 0 1 0 1 1 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 1 0 0 0 1 1]
 [1 0 0 1 0 0 1 0 0 1 0 0 1 0 0 1 0 0 0 1 1 0 1 0 1 1 0 0]
 [1 0 0 0 0 0 0 0 0 1 1 0 0 1 1 0 1 0 0 1 1 0 1 1 0 1 0 0]
 [0 1 0 1 0 1 1 0 1 1 0 1 1 0 1 1 0 0 0 1 1 0 0 1 1 0 0 1]
 [1 0 0 0 1 1 0 0 0 0 1 1 0 0 1 1 0 1 1 0 0 1 0 0 1 0 1 0]
 [1 1 0 0 0 1 0 1 1 0 0 1 0 0 1 0 1 1 0 1 0 0 0 0 1 0 1 0]
 [1 0 1 0 0 0 0 1 1 0 1 0 1 1 0 1 0 0 0 1 0 1 0 1 1 0 0 0]
 [1 0 1 1 0 1 0 0 1 1 0 1 0 0 0 1 1 0 0 0 1 0 0 0 0 0 1 1]
 [0 0 1 1 0 1 1 0 1 0 1 1 0 1 0 0 1 0 0 0 0 1 0 1 1 0 0 0]
 [0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 1 1 0 1]
 [1 1 0 1 0 0 0 1 0 1 1 0 0 0 0 0 0 1 0 0 0 1 0 0 1 0 1 0]
 [0 1 0 0 0 0 0 0 0 1 0 0 1 1 0 1 1 0 0 0 0 1 0 1 0 0 0 0]
 [0 1 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 1 0 0]
 [0 1 1 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0]
 [0 0 1 1 0 0 0 0 1 1 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0]
 [

#Exercici 2: Funció Fitness
Objectiu de la funció és mesurar com de bona és la configuració del resultat. En aquest cas l'objectiu és minimitzar aquest fitness per reduir les penalitzacions i per tant aconseguir un millor individu.
Per avauluar-ho cal tenir en compte el límits d'influència per evitar sobrecàrregues als orbitals. La penalització en cas de no complir-se aquesta condició serà de +5.
D'altra banda també cal penalizar que un quark no pugui influenciar tres orbitals consecutius. En aquesta altra casuística també es penalitzarà amb +10.

In [ ]:
def fitness(adn):
    penalty = 0
    max_influence = [8,8,8,8,10,6,4] * 4
    orbitals_count = np.sum(adn, axis=0)

    for i in range(28):
        if orbitals_count[i] > max_influence[i]:
            penalty += (orbitals_count[i] - max_influence[i]) * 5
    for quark in range (16):
        for orbital in range(26):
            if adn[quark, orbital] == 1 and adn[quark,orbital+1] == 1 and adn[quark, orbital+2] == 1:
                penalitzacio += 10

    return penalty

fitness_value = fitness(initial)
print(fitness_value)

0


# Exercici 3: Creació de la matching pool
Procediment de selecció elitista:
S'agafen els millors individus i que per tant tenen el fitness més baix mentre la resta de la població es descarta.
D'aquesta manera assegurem evitar que per atzar es descartin individus bons i com que en aquest cas hi ha poca variància en les penalitzacions un mètode de ruleta seria gariebé aleatòri.


In [4]:
def elitist_selection(population, fitness_values, elite_size):
    selected_indices = np.argsort(fitness_values)
    best_indices = selected_indices[:elite_size]
    matching_pool = [population[i] for i in best_indices]
    return matching_pool

# Exercici 4: Crossover i mutació

Procediment de reproducció: Crossover
A cada quark es decideix per fila si agafar-la del pare 1 o pare 2 amb probabilitat 50%. Deixant de banda la simplicitat de la informació aquest mètode permet barrejar inforació dels dos pares de manera equilibrada i eviten descompensacions que podrien provocar perdre una estructura útil.

Respecte a la mutació després del crossover s'ha decidit afegir una probabilitat de l'1% de mutar, per tant invertir el valor. Com la matriu és de 16x28 = 446 l'1% serien entre 4 i 5 mutacions a la matriu. D'aquesta manera, s'aconsegueixen introduir petites variacions aleatòries i puntuals evintant l'estancament però sense renunciar a l'estructura que va prenent forma la matriu.

In [5]:
def crossover_5050(parent1, parent2):
    child = np.zeros_like(parent1)
    for quark in range(16):
        if np.random.rand() < 0.5:
            child[quark] = parent1[quark]
        else:
            child[quark] = parent2[quark]
    return child

def mutate(adn, mutation_rate=0.01):
    for quark in range(16):
        for orbital in range(28):
            if np.random.rand() < mutation_rate:
                adn[quark, orbital] = 1 - adn[quark, orbital]
    return adn

In [7]:
pygame.init()

# Constants
WIDTH = 1600
HEIGHT = 900
RADI_ORBITAL = 30
COLOR_ORBITAL = (100, 150, 255)
COLOR_TEXT = (255, 255, 255)
NUM_POPULATION = 10
FPS = 2  

# Pygame inicialització
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Visualització Matriu LS-28")
font = pygame.font.SysFont(None, 24)
clock = pygame.time.Clock()


def draw(populations, fitness_values):
    screen.fill((20, 20, 30))
    marge_x = WIDTH // (28 + 1)
    marge_y = HEIGHT // (len(populations) + 1)
   
    for index, population in enumerate(populations):

        posicions = []
        y = (index + 1) * marge_y
        for i in range(28):
            x = (i + 1) * marge_x
            posicions.append((x, y))

        
        for idx, (x, y) in enumerate(posicions):
            pygame.draw.circle(screen, COLOR_ORBITAL, (x, y), RADI_ORBITAL, 2)
          
            num_quarks = np.sum(population[:, idx])
            if num_quarks > 0:
                text = font.render(str(num_quarks), True, COLOR_TEXT)
                text_rect = text.get_rect(center=(x, y))
                screen.blit(text, text_rect)
        
        fitness_text = font.render(f"Fitness: {fitness_values[index]:.2f}", True, COLOR_TEXT)
        fitness_rect = fitness_text.get_rect(midleft=(WIDTH/2, y-RADI_ORBITAL*1.5))
        screen.blit(fitness_text, fitness_rect)

    fitness_avg = np.mean(fitness_values)
    fitness_avg_text = font.render(f"Fitness Avg: {fitness_avg:.2f}", True, COLOR_TEXT)
    fitness_avg_rect = fitness_avg_text.get_rect(midleft=(WIDTH/2, HEIGHT-RADI_ORBITAL))
    screen.blit(fitness_avg_text, fitness_avg_rect)

    pygame.display.update()

#Inicialitzar la matriu
population = [create_adn() for _ in range(NUM_POPULATION)]
# Bucle principal
running = True
while running:
    clock.tick(FPS)

    for event in pygame.event.get():
        if event.type == pygame.QUIT or (event.type == pygame.KEYDOWN and event.key == pygame.K_ESCAPE):
            running = False

    fitness_values = [fitness(adn) for adn in population]

    parents = elitist_selection(population, fitness_values, elite_size=4)
    childrenPopulation = []
    while len(childrenPopulation) < NUM_POPULATION:
        parent1 = random.choice(parents)
        parent2 = random.choice(parents)
        child = crossover_5050(parent1, parent2)
        child = mutate(child, mutation_rate=0.01)
        if len(childrenPopulation) < NUM_POPULATION:
            childrenPopulation.append(child)

    population = childrenPopulation
    # Dibuixar nova configuració
    draw(population, fitness_values)

pygame.quit()
